In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("Churn_Modelling.csv")
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
# first 3 features have no importance 
df.drop(columns=["RowNumber","CustomerId","Surname"],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# convert gender cat to numerical
print(df["Gender"].unique())
print(df["Gender"].value_counts())
encoder = LabelEncoder()
df["Gender"]=encoder.fit_transform(df["Gender"])
df["Gender"].value_counts()


['Female' 'Male']
Gender
Male      5457
Female    4543
Name: count, dtype: int64


Gender
1    5457
0    4543
Name: count, dtype: int64

In [10]:
# geo feature has more than 2 categories so using one hot encoding here
print(df["Geography"].value_counts())
ohe_encoder = OneHotEncoder(sparse_output=False)
encoded_data = ohe_encoder.fit_transform(df[["Geography"]])
encoded_data


Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
print(ohe_encoder.get_feature_names_out(["Geography"]))
print(len(encoded_data))

['Geography_France' 'Geography_Germany' 'Geography_Spain']
10000


In [12]:
df_ohe_data = pd.DataFrame(encoded_data,columns= ohe_encoder.get_feature_names_out(['Geography']))
df_ohe_data.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [13]:
df = df.drop(columns=["Geography"])
df.reset_index(drop=True, inplace=True)
df_ohe_data.reset_index(drop=True, inplace=True)
data = pd.concat([df,df_ohe_data],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [14]:
# To save encoders and scaler to pickle file 
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(encoder,file)

with open("ohe_encoder.pkl",'wb') as file:
    pickle.dump(ohe_encoder,file)



In [15]:
# divide data and scale the data using standrad scaler
Y = data["Exited"]
X = data.drop(columns=["Exited"],axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state=42,test_size=0.2)

# scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
with open("data_scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

### using tensorflow

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

In [18]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## hidden layer onw connected with input
    Dense(32,activation='relu'), ##Hidden layer 2
    Dense(1,activation='sigmoid') # output layer
])

In [19]:
# to find how many trainable parameters total
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# adam optimizer has fixed learning rate parameter so need to initialize pre
opti = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
# we can pre intialize and use these in compile the model or we can directly initialize in the model itself

In [22]:
# to do forward and back propogation we need to complie the model
model.compile(optimizer=opti,loss="binary_crossentropy",metrics=["accuracy"])

### Setting up the tensor board  before starting the training

In [23]:
# create new ditrectory to store the all the logs while training
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%M%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
# early stop: when training we can train for n number of epochs, if we want to restrict the model based on loss value
earlystopping_callback = EarlyStopping(monitor='var_loss',patience=10,restore_best_weights=True)
# wait till the 10 epochs to run and after that if no further decrease of loss then stop the model

### Training the model

In [26]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                    callbacks=[earlystopping_callback,tensorflow_callback])

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 0.3984 - accuracy: 0.8384 - val_loss: 0.3564 - val_accuracy: 0.8530
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3534 - accuracy: 0.8564 - val_loss: 0.3534 - val_accuracy: 0.8545
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3469 - accuracy: 0.8600 - val_loss: 0.3448 - val_accuracy: 0.8525
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3456 - accuracy: 0.8591 - val_loss: 0.3451 - val_accuracy: 0.8570
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3390 - accuracy: 0.8644 - val_loss: 0.3404 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3390 - accuracy: 0.8622 - val_loss: 0.3416 - val_accuracy: 0.8615
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3372 - accuracy: 0.8620 - val_loss: 0.3404 - val_accuracy: 0.86

In [27]:
model.save('model1.h5')

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# to load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [35]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11280), started 0:00:09 ago. (Use '!kill 11280' to kill it.)